In [2]:
import numpy as np
import pandas as pd
import gc

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train["id"] = -1
test["target"] = -1
data = train.append(test)
del train
del test
gc.collect()

33

In [5]:
data["user_id"] = data["msno"].astype("category").cat.codes.copy()
data["item_id"] = data["song_id"].astype("category").cat.codes.copy()
data.drop(["msno", "song_id"], inplace=True)
gc.collect()
train_data = data[data["id"] == -1][["user_id", "item_id", "target"]]
test_data = data[data["id"] != -1][["user_id", "item_id", "id"]]
del data
gc.collect()

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


56

In [6]:
from sklearn.model_selection import train_test_split

train_train, train_validation = train_test_split(train_data, train_size=0.8)
del train_data
gc.collect()

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


12

In [7]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix

sparse_train = coo_matrix((
    train_train["target"].astype(np.float32),
    (
        train_train["user_id"],
        train_train["item_id"]
    )
))
sample_weight = coo_matrix((
    np.ones(train_train.shape[0]),
    (
        train_train["user_id"],
        train_train["item_id"]
    )
))

In [8]:
# функция, которая красиво печатает информацию о разреженных матрицах

def sparse_info(sparse_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [9]:
sparse_info(sparse_train)

Размерности матрицы: (34403, 419839)
Ненулевых элементов в матрице: 5901934
Доля ненулевых элементов: 0.0004086158928292744
Среднее значение ненулевых элементов: 0.5034465193748474
Максимальное значение ненулевых элементов: 1.0
Минимальное значение ненулевых элементов: 0.0


In [15]:
# добавляем дополнительные фичи
from scipy.sparse import hstack, identity

item_features = hstack([
    identity(train_train.shape[0]),
])

In [16]:
import time
from lightfm import LightFM

model = LightFM(loss="warp")
print("строим модель LightFM")
start = time.time()
model.fit(
    interactions=sparse_train,
    sample_weight=sample_weight,
    num_threads=4,
    epochs=100,
    verbose=True,
    item_features=item_features
)
print("построили LightFM за {} секунд".format(time.time() - start))

/anaconda3/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


строим модель LightFM
Epoch 0
Epoch 1
Epoch 2


KeyboardInterrupt: 

In [ ]:
def get_recs(dataset, item_features):
    print("получаем рекомендации")
    start = time.time()
    recs = model.predict(
        user_ids=np.array(dataset["user_id"].astype(np.int32).tolist()),
        item_ids=np.array(dataset["item_id"].astype(np.int32).tolist()),
        item_features=item_features,
        num_threads=4
    )
    print("получили рекомендации за {} секунд".format(time.time() - start))
    return recs

In [ ]:
from sklearn.metrics import roc_auc_score

train_recs = get_recs(train_train, item_features)
print("train AUC: {}".format(roc_auc_score(train_train["target"], train_recs)))
validation_recs = get_recs(train_validation, item_features)
print("test AUC: {}".format(roc_auc_score(train_validation["target"], validation_recs)))